<a href="https://colab.research.google.com/github/twelvesense/first-repository/blob/master/ExplNode06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 프로젝트: 

# 1 데이터 준비 

## 1-1. 필요한 라이브러리 불러오기

In [1]:
import os, re 
import glob
import tensorflow as tf
# from tensorflow import keras
# import numpy as np

In [2]:
import warnings
warnings.filterwarnings("ignore")
import os, re
from PIL import Image
import glob
import numpy as np
from os.path import join
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

## 1-2. corpus 파일 download하기

In [ ]:
!rm -r /content/lyrics
!mkdir -p /content/lyrics
!wget -O /content/lyrics.zip https://raw.githubusercontent.com/twelvesense/first-repository/master/data/lyrics/lyrics.zip
!unzip /content/lyrics.zip -d /content/lyrics
!rm /content/lyrics.zip

In [5]:
txt_file_path = '/content/lyrics/*'
txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['They say get ready for the revolution', "I think it's time we find some sorta solution", "Somebody's caught up in the endless pollution"]


In [10]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor) 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated        

In [7]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
tensor, tokenizer = tokenize(corpus)
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=22)

[[   2   45   68 ...    0    0    0]
 [   2    5  126 ...    0    0    0]
 [   2  265   16 ...    0    0    0]
 ...
 [   2   19  155 ...    0    0    0]
 [   2    5   32 ...    0    0    0]
 [   2 1133   10 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fd235782a90>


In [9]:
embedding_size = 15
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

#Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [13]:
history = []
epochs = 10
optimizer = tf.keras.optimizers.Adam()
model.compile(loss=loss, optimizer=optimizer)

BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

In [ ]:
model.summary()

In [12]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love rubies bloodstains ceased ceased busy busy cab cab pearls pearls had had sp sp steady steady steady '